In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


In [46]:
def import_data():
    df = pd.read_csv("dataset/1/train.csv")
  
    x=df[["age", "workclass", "fnlwgt", "educational-num", "gender","hours-per-week"]]
    y=df['income_>50K']
    return x, y

input_dataset, output_dataset = import_data()

In [48]:
input_dataset['gender'] = LabelEncoder().fit_transform(input_dataset['gender'])
input_dataset["workclass"] = LabelEncoder().fit_transform(input_dataset["workclass"].astype(str))
SS = StandardScaler()
input_dataset = SS.fit_transform(input_dataset)
x_train, x_test, y_train, y_test = train_test_split( input_dataset, output_dataset, test_size = 0.3, random_state = 0) 


In [50]:

# y_train = tf.keras.utils.to_categorical(y_train)
# y_test = tf.keras.utils.to_categorical(y_test)
y_train= y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)

y_test.shape

(13188, 1)

In [54]:
layers = {
    'input' : 6,
    'hidden' : 4,
    'output' : 1
}

input_hidden = {
    "weight": tf.Variable(tf.random.normal([layers['input'], layers['hidden']])),
    "bias": tf.Variable(tf.random.normal([layers['hidden']]))
}

hidden_output = {
    "weight": tf.Variable(tf.random.normal([layers['hidden'], layers['output']])),
    "bias": tf.Variable(tf.random.normal([layers['output']]))
}

def activation(output):
    return tf.nn.sigmoid(output)

def feed_forward(input_data):
    # Input ke Hidden
    x1 = tf.matmul(input_data, input_hidden["weight"] + input_hidden["bias"])
    y1 = activation(x1)

    # Hidden ke Output
    x2 = tf.matmul(y1, hidden_output["weight"] + hidden_output["bias"])
    y2 = activation(x2)

    return y2, y1

# Placeholder
input_placeholder = tf.placeholder(tf.float32, [None, layers['input']])
output_placeholder = tf.placeholder(tf.float32, [None, layers['output']])

output, hiddenOut = feed_forward(input_placeholder)
error = tf.reduce_mean(0.5 * (output_placeholder - output)** 2)

learning_rate = 0.3
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

epoch = 1000

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    hidden_dict = {
        input_placeholder   : x_train,
        output_placeholder  : y_train
    }

    print(f'"Hidden": {sess.run(hiddenOut, feed_dict = hidden_dict) * 100} %\n')
    tf.summary.scalar('Loss', error)
    tf.summary.histogram('Weight_to_Hidden', input_hidden['weight'])
    

    writer = tf.summary.FileWriter(logdir = './summary')
    summary = tf.summary.merge_all()

    for i in range(epoch):

        train_dict = {
            input_placeholder : x_train,
            output_placeholder : y_train
        }

        sess.run(train, feed_dict = train_dict)
        loss = sess.run(error, feed_dict = train_dict)
        
        if i % 200 == 0:
            print(f"Epoch: {i}, Loss: {loss}") 

    accuracy = tf.equal(tf.argmax(output_placeholder, axis = 1), tf.argmax(output, axis = 1))
    result = tf.reduce_mean(tf.cast(accuracy, tf.float32))
    
    test_dict = {
        input_placeholder : x_test,
        output_placeholder : y_test
    }

    print("Accuracy : {}%\n" .format(sess.run(result, feed_dict = test_dict) * 100))

"Hidden": [[83.72302   98.72135   87.30024   68.09923  ]
 [ 2.3440747 25.906784  38.97002    4.995641 ]
 [ 4.371503  33.13823   39.40101   24.23056  ]
 ...
 [99.75388   22.181889  84.84568   94.15906  ]
 [94.89818   44.15198   97.4273    97.29086  ]
 [48.241573  10.181141  24.07446   80.37952  ]] %

Epoch: 0, Loss: 0.11551646888256073
Epoch: 200, Loss: 0.0869484692811966
Epoch: 400, Loss: 0.07828288525342941
Epoch: 600, Loss: 0.07483465224504471
Epoch: 800, Loss: 0.07298552244901657
Accuracy : 100.0%

